In [0]:
# implement kmeans to classify mnist digits

from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

import os
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x=mnist.train.images

W0810 14:21:53.664062 140178678978432 deprecation.py:323] From <ipython-input-2-07b4fdc0ea89>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0810 14:21:53.669245 140178678978432 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0810 14:21:53.670386 140178678978432 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0810 14:21:54.727560 140178678978432 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0810 14:21:54.731457 140178678978432 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz


W0810 14:21:55.147274 140178678978432 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# parameters
# total steps to train
num_steps=150
# number of samples per batch
batch_size=1024
# number of clusters
k=25
# the 10 digits
num_classes=10
# each image is 28x28 = 784 pixels
num_features=784

# input images
X=tf.placeholder(tf.float32, shape=[None, num_features])
# labels
Y=tf.placeholder(tf.float32, shape=[None, num_classes])

# K-means parameters
kmeans=KMeans(inputs=X, num_clusters=k, distance_metric='cosine', use_mini_batch=True)

In [4]:
# build kmeans graph
all_scores, cluster_idx, scores,  cluster_centers_initialized, init_op, train_op=kmeans.training_graph()

# fix for cluster index being a tuple
cluster_idx=cluster_idx[0]

avg_distance=tf.reduce_mean(scores)

# Initialize variables( i.e assign default values)

init_vars=tf.global_variables_initializer()

W0810 14:21:55.470135 140178678978432 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/factorization/python/ops/clustering_ops.py:740: calling l2_normalize (from tensorflow.python.ops.nn_impl) with dim is deprecated and will be removed in a future version.
Instructions for updating:
dim is deprecated, use axis instead


In [12]:
# start tf session
with tf.Session() as sess:
  sess.run(init_vars, feed_dict={X:full_data_x})
  sess.run(init_op, feed_dict={X:full_data_x})
  
  for i in range(1, num_steps+1):
    _,d,idx=sess.run([train_op, avg_distance, cluster_idx], feed_dict={X:full_data_x})
    
    if i%10==0 or i==1:
      print("Steps %i, Avg distances:%f"%(i,d))
      
    # assign label to each centroid

  # count total number of labels per centroid using label of each training sample
  # to their closest centroid
  count=np.zeros(shape=(k,num_classes))

  for i in range(len(idx)):
    count[idx[i]]+=mnist.train.labels[i]

  # assign most frequent label to centroid
  labels_map=[np.argmax(c) for c in count]
  labels_map=tf.convert_to_tensor(labels_map)

  # evaluation ops
  # lookup: centroid id -> label
  cluster_label=tf.nn.embedding_lookup(labels_map,cluster_idx)

  # compute accuracy
  correct_prediction=tf.equal(cluster_label, tf.cast(tf.argmax(Y,1), tf.int32))
  accuracy_op=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  test_x, test_y=mnist.test.images, mnist.test.labels
  print("Test accuracy:",sess.run(accuracy_op,feed_dict={X:test_x, Y:test_y}))

Steps 1, Avg distances:0.341471
Steps 10, Avg distances:0.221609
Steps 20, Avg distances:0.220328
Steps 30, Avg distances:0.219776
Steps 40, Avg distances:0.219419
Steps 50, Avg distances:0.219154
Steps 60, Avg distances:0.218940
Steps 70, Avg distances:0.218764
Steps 80, Avg distances:0.218614
Steps 90, Avg distances:0.218485
Steps 100, Avg distances:0.218373
Steps 110, Avg distances:0.218276
Steps 120, Avg distances:0.218190
Steps 130, Avg distances:0.218113
Steps 140, Avg distances:0.218043
Steps 150, Avg distances:0.217978
Test accuracy: 0.7315
